In [1]:
import pandas as pd
import os
import math
import time
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import traceback
import cv2
import subprocess
import shutil
from moviepy.video.io.VideoFileClip import VideoFileClip
import mediapipe as mp
import numpy as np

## Step 1: Separate Left and Right Hand Video Segments

In [2]:
def create_directory(file_path):
    '''
    Create a directory if it does not exist.
    Create parent directories as well if necessary.
    '''
    if os.path.exists(file_path):
        return
    
    parent = os.path.dirname(file_path)
    
    if os.path.exists(parent):
        os.mkdir(file_path)
        return
    
    create_directory(parent)
    os.mkdir(file_path)
    return

def remove_and_clear_directory(file_path):
    '''
    Delete this directory, with all subdirectories and contents
    '''
    shutil.rmtree(file_path)

def number_to_time(time_in_fraction):
    '''
    input: 0.45 = 45 seconds
    output: 00:00:45 (hh:mm:ss)
    '''
    seconds = round(time_in_fraction*100,2)
    assert(seconds<60)
    if seconds<10:
        time = "00:00:0"+str(seconds)
    else:
        time = "00:00:"+str(seconds)
    return time

In [3]:
def video_properties(file_path):
    '''
    Given a file path, returns the following:
    duration (in seconds)
    '''
    vidcapture = cv2.VideoCapture(file_path)
    fps = vidcapture.get(cv2.CAP_PROP_FPS)
    totalNoFrames = vidcapture.get(cv2.CAP_PROP_FRAME_COUNT);
    durationInSeconds = float(totalNoFrames) / float(fps)
    return durationInSeconds

In [4]:
def process_one_video_pose(video_path, output_path):
    '''
    Temporal segmentation of a single video
    '''
    from matplotlib.patches import Circle
    from skimage import io
    import matplotlib.pyplot as plt
    
    VISIBILITY_THR = 0.60
    
    visible_frames = []
    frame_no = 1
    
    cap = cv2.VideoCapture(video_path)
    with mp_pose.Pose(
        static_image_mode=False,
        min_detection_confidence=0.5,
        upper_body_only=True) as pose:
        
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                break

            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = pose.process(image)
            patches = []
            
            # Print nose landmark.
            image_hight, image_width, _ = image.shape
            if not results.pose_landmarks:
                continue
                
            (LEFT_WRIST_X, LEFT_WRIST_Y) = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].x * image_width, results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y * image_hight)
            (RIGHT_WRIST_X, RIGHT_WRIST_Y) = (results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].x * image_width, results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y * image_hight)
            
            if results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].visibility > VISIBILITY_THR \
            or results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].visibility > VISIBILITY_THR:
                visible_frames.append(frame_no)
                
            frame_no +=1
        
            if cv2.waitKey(5) & 0xFF == 27:
                break
        
        cap.release()
        cv2.destroyAllWindows()
        
    #print(visible_frames)
    visible_frames = np.asarray(visible_frames)
    first_frame_no = np.min(visible_frames)
    last_frame_no = np.max(visible_frames)
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    video_out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'MJPG'),15, (frame_width, frame_height))

    frame_no = 1
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            break

        if frame_no>=first_frame_no and frame_no<=last_frame_no:
            video_out.write(image)

        frame_no +=1
        if cv2.waitKey(5) & 0xFF == 27:
            break

    cap.release()
    video_out.release()
    cv2.destroyAllWindows()

In [6]:
#Read Excel Sheet
INPUT_LR_SPLIT_FILE = "E:/Saiful/park_motor/LeftRight/data/right_left_label.csv"
#Original Task2 Videos
INPUT_TASK2_VIDEOS = "E:/Saiful/park_motor/Task2_All"

#remove_and_clear_directory("E:/Saiful/park_motor/LeftRight/Task2_Videos/")

#This is where separated videos will go
OUTPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left"
OUTPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right"

#Create directories (full path if needed) if not present
create_directory(OUTPUT_LEFT_VIDEOS)
create_directory(OUTPUT_RIGHT_VIDEOS)

error_files = []

#temp_dir = os.path.join(OUTPUT_LEFT_VIDEOS, "temp")
#create_directory(temp_dir)

### 1.1 Time crop the videos and save as webm

In [7]:
success = 0
failed = 0

test_count = 0

df = pd.read_csv(INPUT_LR_SPLIT_FILE)
for index, row in df.iterrows():
    
    filename = row["File_name"]
    left_start = row["left_begin"]
    left_end = row["left_end"]
    right_start = row[" right_begin"]
    right_end = row["right_end"]
    
    print("Left %f to %f, Right %f to %f"%(left_start, left_end, right_start, right_end))
    
    input_file_path = os.path.join(INPUT_TASK2_VIDEOS, filename)
    #Make sure it exists
    assert(os.path.exists(input_file_path))
    
    output_left_filename = filename[:-5]+"-left"+".webm"
    output_right_filename = filename[:-5]+"-right"+".webm"
    
    output_left_path = os.path.join(OUTPUT_LEFT_VIDEOS, output_left_filename)
    output_right_path = os.path.join(OUTPUT_RIGHT_VIDEOS, output_right_filename)
    
    if (left_start!=(-1.0)) and (left_end!=(-1.0)):
        #Clip left part
        if not os.path.exists(output_left_path):
            #left_start_time = number_to_time(left_start)
            #left_end_time = number_to_time(left_end)
            left_start_time = left_start*100
            left_end_time = (left_end*100)
            
            command = "ffmpeg -i \"%s\" -ss %f -t %f \"%s\""%(input_file_path, left_start_time, (left_end_time - left_start_time), output_left_path)
            out = os.system(command)
            print(out)
            if out==0:
                time.sleep(2)
                success +=1
            else:
                failed +=1
                error_files.append(output_left_path)

    if (right_start!=(-1.0)) and (right_end!=(-1.0)):
        #Clip right part
        if not os.path.exists(output_right_path):
            #right_start_time = number_to_time(right_start)
            #right_end_time = number_to_time(right_end)
            right_start_time = right_start*100
            right_end_time = (right_end*100)
            
            #ffmpeg -i "source_path" --ss start_time_second -t duration_second "destination_path"
            command = "ffmpeg -i \"%s\" -ss %f -t %f \"%s\""%(input_file_path, right_start_time, (right_end_time - right_start_time), output_right_path)
            out = os.system(command)
            print(out)
            if out==0:
                time.sleep(2)
                success +=1
            else:
                failed +=1
                error_files.append(output_left_path)
                
    print("Attempted %d, success %d, failed %d \n"%((failed+success), success, failed))

Left 0.020000 to 0.080000, Right 0.080000 to 0.160000
0
0
Attempted 2, success 2, failed 0 

Left 0.110000 to 0.280000, Right 0.000000 to 0.110000
0
0
Attempted 4, success 4, failed 0 

Left 0.000000 to 0.050000, Right 0.050000 to 0.110000
0
0
Attempted 6, success 6, failed 0 

Left 0.040000 to 0.080000, Right 0.000000 to 0.040000
0
0
Attempted 8, success 8, failed 0 

Left 0.030000 to 0.090000, Right 0.000000 to 0.030000
0
0
Attempted 10, success 10, failed 0 

Left 0.040000 to 0.110000, Right 0.000000 to 0.040000
0
0
Attempted 12, success 12, failed 0 

Left 0.000000 to 0.050000, Right 0.050000 to 0.110000
0
0
Attempted 14, success 14, failed 0 

Left 0.030000 to 0.090000, Right 0.000000 to 0.030000
0
0
Attempted 16, success 16, failed 0 

Left -1.000000 to -1.000000, Right 0.000000 to 0.120000
0
Attempted 17, success 17, failed 0 

Left 0.000000 to 0.040000, Right 0.040000 to 0.090000
0
0
Attempted 19, success 19, failed 0 

Left 0.040000 to 0.110000, Right 0.000000 to 0.040000
0
0


0
Attempted 170, success 170, failed 0 

Left 0.060000 to 0.130000, Right 0.000000 to 0.060000
0
0
Attempted 172, success 172, failed 0 

Left 0.050000 to 0.110000, Right 0.000000 to 0.050000
0
0
Attempted 174, success 174, failed 0 

Left 0.040000 to 0.100000, Right 0.000000 to 0.040000
0
0
Attempted 176, success 176, failed 0 

Left 0.000000 to 0.050000, Right 0.050000 to 0.140000
0
0
Attempted 178, success 178, failed 0 

Left 0.030000 to 0.080000, Right 0.000000 to 0.030000
0
0
Attempted 180, success 180, failed 0 

Left 0.000000 to 0.080000, Right 0.080000 to 0.180000
0
0
Attempted 182, success 182, failed 0 

Left -1.000000 to -1.000000, Right -1.000000 to -1.000000
Attempted 182, success 182, failed 0 

Left 0.000000 to 0.050000, Right 0.050000 to 0.100000
0
0
Attempted 184, success 184, failed 0 

Left 0.035000 to 0.080000, Right 0.000000 to 0.035000
0
0
Attempted 186, success 186, failed 0 

Left 0.050000 to 0.110000, Right 0.000000 to 0.050000
0
0
Attempted 188, success 188, 

0
0
Attempted 334, success 334, failed 0 

Left 0.100000 to 0.240000, Right 0.000000 to 0.100000
0
0
Attempted 336, success 336, failed 0 

Left 0.000000 to 0.050000, Right 0.040000 to 0.100000
0
0
Attempted 338, success 338, failed 0 

Left 0.000000 to 0.030000, Right 0.030000 to 0.060000
0
0
Attempted 340, success 340, failed 0 

Left 0.030000 to 0.080000, Right 0.000000 to 0.030000
0
0
Attempted 342, success 342, failed 0 

Left 0.040000 to 0.100000, Right 0.000000 to 0.040000
0
0
Attempted 344, success 344, failed 0 

Left 0.050000 to 0.140000, Right -1.000000 to -1.000000
0
Attempted 345, success 345, failed 0 

Left 0.060000 to 0.140000, Right 0.000000 to 0.060000
0
0
Attempted 347, success 347, failed 0 

Left 0.000000 to 0.020000, Right 0.020000 to 0.100000
0
0
Attempted 349, success 349, failed 0 

Left -1.000000 to -1.000000, Right -1.000000 to -1.000000
Attempted 349, success 349, failed 0 

Left 0.040000 to 0.100000, Right 0.000000 to 0.040000
0
0
Attempted 351, success 351

0
0
Attempted 496, success 496, failed 0 

Left 0.000000 to 0.035000, Right 0.035000 to 0.080000
0
0
Attempted 498, success 498, failed 0 

Left 0.000000 to 0.035000, Right 0.035000 to 0.080000
0
0
Attempted 500, success 500, failed 0 

Left 0.000000 to 0.050000, Right 0.050000 to 0.100000
0
0
Attempted 502, success 502, failed 0 

Left 0.000000 to 0.050000, Right 0.045000 to 0.100000
0
0
Attempted 504, success 504, failed 0 

Left 0.000000 to 0.050000, Right 0.050000 to 0.110000
0
0
Attempted 506, success 506, failed 0 

Left 0.035000 to 0.080000, Right 0.000000 to 0.040000
0
0
Attempted 508, success 508, failed 0 

Left 0.050000 to 0.130000, Right 0.000000 to 0.050000
0
0
Attempted 510, success 510, failed 0 

Left 0.110000 to 0.160000, Right 0.000000 to 0.110000
0
0
Attempted 512, success 512, failed 0 

Left 0.000000 to 0.040000, Right 0.040000 to 0.110000
0
0
Attempted 514, success 514, failed 0 

Left 0.040000 to 0.090000, Right 0.000000 to 0.040000
0
0
Attempted 516, success 516

0
0
Attempted 665, success 665, failed 0 

Left 0.000000 to 0.050000, Right 0.050000 to 0.100000
0
0
Attempted 667, success 667, failed 0 

Left 0.050000 to 0.110000, Right 0.000000 to 0.050000
0
0
Attempted 669, success 669, failed 0 

Left 0.060000 to 0.150000, Right 0.000000 to 0.060000
0
0
Attempted 671, success 671, failed 0 

Left 0.080000 to 0.170000, Right 0.000000 to 0.090000
0
0
Attempted 673, success 673, failed 0 

Left 0.060000 to 0.130000, Right 0.000000 to 0.060000
0
0
Attempted 675, success 675, failed 0 

Left 0.000000 to 0.020000, Right 0.020000 to 0.050000
0
0
Attempted 677, success 677, failed 0 

Left 0.025000 to 0.070000, Right 0.000000 to 0.030000
0
0
Attempted 679, success 679, failed 0 

Left -1.000000 to -1.000000, Right 0.000000 to 0.090000
0
Attempted 680, success 680, failed 0 

Left 0.030000 to 0.100000, Right 0.000000 to 0.030000
0
0
Attempted 682, success 682, failed 0 

Left 0.000000 to 0.040000, Right 0.040000 to 0.100000
0
0
Attempted 684, success 684

0
0
Attempted 828, success 828, failed 0 

Left 0.030000 to 0.060000, Right 0.000000 to 0.030000
0
0
Attempted 830, success 830, failed 0 

Left 0.045000 to 0.090000, Right 0.000000 to 0.050000
0
0
Attempted 832, success 832, failed 0 

Left 0.060000 to 0.120000, Right 0.000000 to 0.060000
0
0
Attempted 834, success 834, failed 0 

Left 0.120000 to 0.290000, Right 0.000000 to 0.120000
0
0
Attempted 836, success 836, failed 0 

Left 0.030000 to 0.090000, Right 0.000000 to 0.030000
0
0
Attempted 838, success 838, failed 0 

Left 0.070000 to 0.170000, Right 0.000000 to 0.070000
0
0
Attempted 840, success 840, failed 0 

Left 0.000000 to 0.070000, Right 0.070000 to 0.150000
0
0
Attempted 842, success 842, failed 0 

Left 0.070000 to 0.160000, Right 0.000000 to 0.070000
0
0
Attempted 844, success 844, failed 0 

Left 0.000000 to 0.075000, Right 0.070000 to 0.130000
0
0
Attempted 846, success 846, failed 0 

Left 0.000000 to 0.070000, Right 0.070000 to 0.230000
0
0
Attempted 848, success 848

0
0
Attempted 988, success 988, failed 0 

Left 0.045000 to 0.120000, Right 0.000000 to 0.050000
0
0
Attempted 990, success 990, failed 0 

Left 0.045000 to 0.110000, Right 0.000000 to 0.050000
0
0
Attempted 992, success 992, failed 0 

Left 0.000000 to 0.070000, Right 0.065000 to 0.120000
0
0
Attempted 994, success 994, failed 0 

Left 0.000000 to 0.040000, Right 0.040000 to 0.090000
0
0
Attempted 996, success 996, failed 0 

Left 0.065000 to 0.170000, Right 0.000000 to 0.070000
0
0
Attempted 998, success 998, failed 0 

Left 0.000000 to 0.050000, Right 0.040000 to 0.090000
0
0
Attempted 1000, success 1000, failed 0 

Left 0.000000 to 0.055000, Right 0.050000 to 0.120000
0
0
Attempted 1002, success 1002, failed 0 

Left 0.000000 to 0.050000, Right 0.050000 to 0.100000
0
0
Attempted 1004, success 1004, failed 0 

Left 0.050000 to 0.110000, Right 0.000000 to 0.050000
0
0
Attempted 1006, success 1006, failed 0 

Left 0.040000 to 0.110000, Right 0.000000 to 0.045000
0
0
Attempted 1008, su

0
0
Attempted 1147, success 1147, failed 0 

Left 0.000000 to 0.030000, Right 0.030000 to 0.070000
0
0
Attempted 1149, success 1149, failed 0 

Left 0.030000 to 0.050000, Right 0.000000 to 0.030000
0
0
Attempted 1151, success 1151, failed 0 

Left 0.080000 to 0.180000, Right 0.000000 to 0.090000
0
0
Attempted 1153, success 1153, failed 0 

Left 0.000000 to 0.040000, Right 0.040000 to 0.090000
0
0
Attempted 1155, success 1155, failed 0 

Left 0.000000 to 0.045000, Right 0.040000 to 0.090000
0
0
Attempted 1157, success 1157, failed 0 

Left 0.050000 to 0.120000, Right 0.000000 to 0.050000
0
0
Attempted 1159, success 1159, failed 0 

Left 0.040000 to 0.110000, Right 0.000000 to 0.045000
0
0
Attempted 1161, success 1161, failed 0 

Left 0.000000 to 0.070000, Right 0.070000 to 0.140000
0
0
Attempted 1163, success 1163, failed 0 

Left 0.070000 to 0.130000, Right 0.000000 to 0.070000
0
0
Attempted 1165, success 1165, failed 0 

Left 0.000000 to 0.060000, Right 0.060000 to 0.140000
0
0
Attemp

0
0
Attempted 1312, success 1312, failed 0 

Left 0.000000 to 0.040000, Right 0.040000 to 0.075000
0
0
Attempted 1314, success 1314, failed 0 

Left 0.045000 to 0.130000, Right 0.000000 to 0.050000
0
0
Attempted 1316, success 1316, failed 0 

Left 0.000000 to 0.050000, Right 0.050000 to 0.140000
0
0
Attempted 1318, success 1318, failed 0 

Left 0.040000 to 0.070000, Right 0.000000 to 0.040000
0
0
Attempted 1320, success 1320, failed 0 

Left 0.000000 to 0.040000, Right 0.035000 to 0.070000
0
0
Attempted 1322, success 1322, failed 0 

Left 0.065000 to 0.120000, Right 0.000000 to 0.070000
0
0
Attempted 1324, success 1324, failed 0 

Left 0.400000 to 0.550000, Right 0.250000 to 0.400000
0
0
Attempted 1326, success 1326, failed 0 

Left 0.000000 to 0.080000, Right 0.075000 to 0.140000
0
0
Attempted 1328, success 1328, failed 0 

Left 0.060000 to 0.180000, Right 0.000000 to 0.070000
0
0
Attempted 1330, success 1330, failed 0 

Left 0.150000 to 0.270000, Right 0.000000 to 0.150000
0
0
Attemp

0
0
Attempted 1478, success 1478, failed 0 

Left 0.050000 to 0.100000, Right 0.000000 to 0.050000
0
0
Attempted 1480, success 1480, failed 0 

Left 0.030000 to 0.080000, Right 0.000000 to 0.035000
0
0
Attempted 1482, success 1482, failed 0 

Left 0.035000 to 0.080000, Right 0.000000 to 0.040000
0
0
Attempted 1484, success 1484, failed 0 

Left 0.050000 to 0.100000, Right 0.000000 to 0.050000
0
0
Attempted 1486, success 1486, failed 0 

Left 0.035000 to 0.080000, Right 0.000000 to 0.040000
0
0
Attempted 1488, success 1488, failed 0 

Left 0.035000 to 0.070000, Right 0.000000 to 0.035000
0
0
Attempted 1490, success 1490, failed 0 

Left 0.135000 to 0.190000, Right 0.000000 to 0.135000
0
0
Attempted 1492, success 1492, failed 0 

Left 0.035000 to 0.090000, Right 0.000000 to 0.040000
0
0
Attempted 1494, success 1494, failed 0 

Left 0.025000 to 0.060000, Right 0.000000 to 0.025000
0
0
Attempted 1496, success 1496, failed 0 

Left 0.025000 to 0.070000, Right 0.000000 to 0.030000
0
0
Attemp

In [8]:
#Errors while time cropping the videos
with open('E:/Saiful/park_motor/LeftRight/errors.txt', 'w') as f:
    for item in error_files:
        f.write("%s\n" % item)

## Step 2: Adjusting Frames per second (FPS = 15), Resize to 256x256, Convert to mp4

    NEEDS REWORK FROM THE BEGINNING OF THIS SECTION

### 2.1 Adjusting Frames per second (FPS = 15)

In [9]:
INPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left"
INPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right"
OUTPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left15"
OUTPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right15"

create_directory(OUTPUT_LEFT_VIDEOS)
create_directory(OUTPUT_RIGHT_VIDEOS)

error_files = []

print("Converting left hand videos\n===============\n")
files = os.listdir(INPUT_LEFT_VIDEOS)
for f in files:
    if ".webm" not in f:
        continue
    input_file = os.path.join(INPUT_LEFT_VIDEOS,f)
    output_file = os.path.join(OUTPUT_LEFT_VIDEOS,f)
    out=0
    
    if not os.path.exists(output_file):
        command = "ffmpeg -y -r 15 -i \"%s\" \"%s\""%(input_file, output_file)
        out = os.system(command)
        print(out)
    
    
    if out!=0:
        print("fps conversion error")
        print(command)
        error_files.append(output_file)


print("Converting right hand videos\n===============\n")
files = os.listdir(INPUT_RIGHT_VIDEOS)
for f in files:
    if ".webm" not in f:
        continue
    input_file = os.path.join(INPUT_RIGHT_VIDEOS,f)
    output_file = os.path.join(OUTPUT_RIGHT_VIDEOS,f)
    out=0
    
    if not os.path.exists(output_file):
        command = "ffmpeg -y -r 15 -i \"%s\" \"%s\""%(input_file, output_file)
        out = os.system(command)
        print(out)
    
    if out!=0:
        print("fps conversion error")
        print(command)
        error_files.append(output_file)
        
with open('E:/Saiful/park_motor/LeftRight/errors_fps_conversion.txt', 'w') as f:
    for item in error_files:
        f.write("%s\n" % item)

Converting left hand videos

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

### 2.2 Converting to mp4 and resizing to 256x256

In [10]:
import os

INPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left15"
INPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right15"

OUTPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left_Resized"
OUTPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right_Resized"

create_directory(OUTPUT_LEFT_VIDEOS)
create_directory(OUTPUT_RIGHT_VIDEOS)

error_files = []

print("Converting left hand videos\n===============\n")
files = os.listdir(INPUT_LEFT_VIDEOS)
for f in files:
    if ".webm" not in f:
        continue
    input_file = os.path.join(INPUT_LEFT_VIDEOS,f)
    output_file = os.path.join(OUTPUT_LEFT_VIDEOS,f[0:-5]+".mp4")
    #output_file = output_folder+f
    #print(input_file)
    #print(output_file)
    
    #command: ffmpeg -i video.webm -c:v copy video.mp4
        
    command = "ffmpeg -i \"%s\" -vf scale=256:256 \"%s\""%(input_file, output_file)
    out = os.system(command)
    print(out)
    
    if out!=0:
        print("mp4 conversion error")
        print(command)
        error_files.append(output_file)
        

print("Converting right hand videos\n===============\n")
files = os.listdir(INPUT_RIGHT_VIDEOS)
for f in files:
    if ".webm" not in f:
        continue
    input_file = os.path.join(INPUT_RIGHT_VIDEOS,f)
    output_file = os.path.join(OUTPUT_RIGHT_VIDEOS,f[0:-5]+".mp4")
    #output_file = output_folder+f
    #print(input_file)
    #print(output_file)
    
    #command: ffmpeg -i video.webm -c:v copy video.mp4
        
    command = "ffmpeg -i \"%s\" -vf scale=256:256 \"%s\""%(input_file, output_file)
    out = os.system(command)
    print(out)
    
    if out!=0:
        print("mp4 conversion error")
        print(command)
        error_files.append(output_file)
        
with open('E:/Saiful/park_motor/LeftRight/errors_mp4_conversion.txt', 'w') as f:
    for item in error_files:
        f.write("%s\n" % item)
    

Converting left hand videos

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

## Step 3: Temporal Segmentation

In [13]:
#Directory setup
INPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Left_Resized"
INPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/Right_Resized"

OUTPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/LeftTemporalSegmentation"
OUTPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/RightTemporalSegmentation"

create_directory(OUTPUT_LEFT_VIDEOS)
create_directory(OUTPUT_RIGHT_VIDEOS)

#Mediapipe global variables
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose

print("Temporal segmentation of the left hand finger tapping\n==============\n")
#List the files
file_list = [filename for filename in os.listdir(INPUT_LEFT_VIDEOS) if '.mp4' in filename]
file_list_fullpath = [os.path.join(INPUT_LEFT_VIDEOS,filename) for filename in file_list]

problems = [] #Track the errors
for file_path in file_list_fullpath:
    try:
        output_path = os.path.join(OUTPUT_LEFT_VIDEOS,file_path[file_path.find("\\")+1:])
        if not os.path.exists(output_path):
            print(output_path)
            process_one_video_pose(file_path, output_path)  
    except:
        print("Error in temporal segmentation")
        print(file_path)
        problems.append(file_path)

print("Temporal segmentation of the right hand finger tapping\n==============\n")
#List the files
file_list = [filename for filename in os.listdir(INPUT_RIGHT_VIDEOS) if '.mp4' in filename]
file_list_fullpath = [os.path.join(INPUT_RIGHT_VIDEOS,filename) for filename in file_list]

for file_path in file_list_fullpath:
    try:
        output_path = os.path.join(OUTPUT_RIGHT_VIDEOS,file_path[file_path.find("\\")+1:])
        if not os.path.exists(output_path):
            print(output_path)
            process_one_video_pose(file_path, output_path)  
    except:
        print("Error in temporal segmentation")
        print(file_path)
        problems.append(file_path)
    
print(problems)

with open('E:/Saiful/park_motor/LeftRight/errors_temporal_segment.txt', 'w') as f:
    for item in problems:
        f.write("%s\n" % item)

Temporal segmentation of the left hand finger tapping

E:/Saiful/park_motor/LeftRight/Task2_Videos/LeftTemporalSegmentation\2018-07-25T21-20-52-024Z12-task2-left.mp4
Ignoring empty camera frame.
Error in temporal segmentation
E:/Saiful/park_motor/LeftRight/Task2_Videos/Left_Resized\2018-07-25T21-20-52-024Z12-task2-left.mp4
E:/Saiful/park_motor/LeftRight/Task2_Videos/LeftTemporalSegmentation\2018-09-20T18-13-34-402Z89-task2-left.mp4
Ignoring empty camera frame.
Error in temporal segmentation
E:/Saiful/park_motor/LeftRight/Task2_Videos/Left_Resized\2018-09-20T18-13-34-402Z89-task2-left.mp4
E:/Saiful/park_motor/LeftRight/Task2_Videos/LeftTemporalSegmentation\2019-10-21T23-39-24-075Z30-task2-left.mp4
Ignoring empty camera frame.
Error in temporal segmentation
E:/Saiful/park_motor/LeftRight/Task2_Videos/Left_Resized\2019-10-21T23-39-24-075Z30-task2-left.mp4
E:/Saiful/park_motor/LeftRight/Task2_Videos/LeftTemporalSegmentation\2020-01-23T19-01-41-966Z28-task2-left.mp4
Ignoring empty camera fr

## Step 4: Spatial Segmentation (BodyPix Background Removal)

In [18]:
import os

INPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/LeftTemporalSegmentation"
INPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/RightTemporalSegmentation"

OUTPUT_LEFT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/LeftBodyPix"
OUTPUT_RIGHT_VIDEOS = "E:/Saiful/park_motor/LeftRight/Task2_Videos/RightBodyPix"

background_file = "E:/Saiful/park_motor/pure-black-background.jpg"

assert(os.path.exists(background_file))

create_directory(OUTPUT_LEFT_VIDEOS)
create_directory(OUTPUT_RIGHT_VIDEOS)

error_files = []

print("Removing Background from Left Videos")
files = os.listdir(INPUT_LEFT_VIDEOS)

for f in files:
    if ".mp4" not in f:
        continue
    input_file = os.path.join(INPUT_LEFT_VIDEOS,f)
    output_file = os.path.join(OUTPUT_LEFT_VIDEOS,"temp.jpg")
    
    if os.path.exists(os.path.join(OUTPUT_LEFT_VIDEOS,f)):
        continue
        
    command = "python -m tf_bodypix replace-background --source \"%s\" --background \"%s\" --output \"%s\" --threshold=0.35 --mask-blur=5"%(input_file, background_file, output_file)
    out = os.system(command)
    if not out==0:
        error_files.append(input_file)
    print(out)
    

print("Removing Background from Right Videos")
files = os.listdir(INPUT_RIGHT_VIDEOS)

for f in files:
    if ".mp4" not in f:
        continue
    input_file = os.path.join(INPUT_RIGHT_VIDEOS,f)
    output_file = os.path.join(OUTPUT_RIGHT_VIDEOS,"temp.jpg")
    
    if os.path.exists(os.path.join(OUTPUT_RIGHT_VIDEOS,f)):
        continue
        
    command = "python -m tf_bodypix replace-background --source \"%s\" --background \"%s\" --output \"%s\" --threshold=0.35 --mask-blur=5"%(input_file, background_file, output_file)
    out = os.system(command)
    if not out==0:
        error_files.append(input_file)
    print(out)


Removing Background from Left Videos
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

NameError: name 'output_folder' is not defined

In [20]:
output_folder = "E:/Saiful/park_motor/LeftRight"
with open(os.path.join(output_folder,'errors_background_removal.txt'), 'w') as f:
    for item in error_files:
        f.write("%s\n" % item)
        
print("Spatial Segmentation Complete")

Spatial Segmentation Complete
